In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [2]:
LOADPATH = '/home/a1r/바탕화면/DL/timeseries_new_data/'
PATH = os.getcwd() + '/data/'
new_train = pd.read_csv(LOADPATH + 'train_fe.csv', low_memory=False)
origin_train = pd.read_csv(PATH + 'train.csv')

In [3]:
new_train.head(3)

,ID,제품,대분류,중분류,소분류,브랜드,date,sales,year,month,day,day_of_week,day_name,quarter,keyword,price,event
0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2022-01-01,0,2022,1,1,5,Saturday,1,0.84131,0,1월 1일
1,1,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,2022-01-01,0,2022,1,1,5,Saturday,1,12.64868,0,1월 1일
2,2,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,2022-01-01,0,2022,1,1,5,Saturday,1,12.64868,0,1월 1일


In [5]:
batch_size = 32 
in_seq_len = 3  # input_channel
pred_len = 2    # output_channel
channel = 5     # channel

X = torch.randn(batch_size, in_seq_len, channel)    # shape = (32, 3, 5)
one_linear = nn.Linear(in_seq_len, pred_len)        # 3 channel로 받아서 2 channel로 출력
output = torch.zeros(batch_size, pred_len, channel) # shape = (32, 2, 5) | input_data인 X와 in/out channel만 바뀜
output[:,:,0]=one_linear(X[:,:,0])
output = one_linear(X.permute(0,2,1)).permute(0,2,1)
print(output)
print('\n', output.shape)

tensor([[[ 1.2883e-01,  3.9684e-01,  5.8147e-01,  8.9197e-01, -1.2178e-01],
         [ 2.6436e-01, -7.2082e-01, -6.1874e-01, -4.3565e-01, -7.5797e-01]],

        [[-3.2123e-02,  4.9942e-01,  2.2893e-01,  3.6523e-01,  9.5176e-01],
         [-5.7484e-01,  7.1410e-01, -5.1304e-01, -7.2099e-01, -2.7029e-01]],

        [[ 8.1165e-01, -1.0705e-01,  4.2821e-01,  8.3535e-01, -6.0847e-01],
         [ 8.0642e-01, -6.3051e-01, -7.4844e-01, -9.5304e-01, -1.1808e+00]],

        [[-7.2356e-01,  5.4580e-01, -4.8588e-01,  2.1391e-01,  9.0223e-01],
         [-1.3511e+00, -9.5176e-01, -7.4209e-01, -4.3952e-01,  8.6004e-01]],

        [[ 5.0669e-01,  3.2170e-01,  6.7561e-01,  4.2196e-01,  5.4540e-01],
         [ 1.0677e-01, -5.7643e-01, -2.3371e-01, -4.6806e-01, -1.8208e-01]],

        [[ 3.1907e-01,  3.3343e-01,  1.3904e-01, -6.6101e-02,  9.4917e-02],
         [-3.5025e-01,  9.9582e-01, -1.0573e+00, -7.2394e-01, -8.1114e-01]],

        [[ 4.6972e-01,  4.9392e-01,  6.6735e-01,  6.8982e-01,  5.2254e-01],


## DLinear Model

In [ ]:
class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size = 25, stride = 1):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
    	# [BATCH SIZE, SEQ_LEN, CHANNEL]
        # padding on the both ends of time series

        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x # [BATCH SIZE, SEQ_LEN, CHANNEL]

In [ ]:
class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

In [ ]:
class Model(nn.Module):
    """
    Decomposition-Linear
    """
    def __init__(self, configs):
        super(Model, self).__init__()
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len

        # Decompsition Kernel Size
        kernel_size = 25
        self.decompsition = series_decomp(kernel_size)
        self.individual = configs.individual
        self.channels = configs.enc_in

        if self.individual:
            self.Linear_Seasonal = nn.ModuleList()
            self.Linear_Trend = nn.ModuleList()
            
            for i in range(self.channels):
                self.Linear_Seasonal.append(nn.Linear(self.seq_len,self.pred_len))
                self.Linear_Trend.append(nn.Linear(self.seq_len,self.pred_len))

                # Use this two lines if you want to visualize the weights
                # self.Linear_Seasonal[i].weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
                # self.Linear_Trend[i].weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
        else:
            self.Linear_Seasonal = nn.Linear(self.seq_len,self.pred_len)
            self.Linear_Trend = nn.Linear(self.seq_len,self.pred_len)
            
            # Use this two lines if you want to visualize the weights
            # self.Linear_Seasonal.weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
            # self.Linear_Trend.weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))

    def forward(self, x):
        # x: [Batch, Input length, Channel]
        seasonal_init, trend_init = self.decompsition(x)
        # seasonal_init: [Batch, Input length, Channel]
        # trend_init: [Batch, Input length, Channel]
        seasonal_init, trend_init = seasonal_init.permute(0,2,1), trend_init.permute(0,2,1)
        # seasonal_init: [Batch, Channel, Input length]
        # trend_init: [Batch, Channel, Input length]
        if self.individual:
            seasonal_output = torch.zeros([seasonal_init.size(0),seasonal_init.size(1),self.pred_len],dtype=seasonal_init.dtype).to(seasonal_init.device)
            trend_output = torch.zeros([trend_init.size(0),trend_init.size(1),self.pred_len],dtype=trend_init.dtype).to(trend_init.device)
            for i in range(self.channels):
                seasonal_output[:,i,:] = self.Linear_Seasonal[i](seasonal_init[:,i,:])
                trend_output[:,i,:] = self.Linear_Trend[i](trend_init[:,i,:])
        else:
            seasonal_output = self.Linear_Seasonal(seasonal_init)
            trend_output = self.Linear_Trend(trend_init)

        x = seasonal_output + trend_output
        return x.permute(0,2,1) # to [Batch, Output length, Channel]